In [1]:
import fire
import json
import torch
from data import GrandStaffDataset
from smt_trainer import SMT_Trainer

from ExperimentConfig import experiment_config_from_dict
from lightning.pytorch import Trainer
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.loggers import WandbLogger
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
import torch
import cv2
from data_augmentation.data_augmentation import convert_img_to_tensor
from smt_model import SMTModelForCausalLM
import os
from weight_transfer import map_lm_to_smt


/opt/conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:

with open("config/GrandStaff/grandstaff.json", "r") as f:
  config = experiment_config_from_dict(json.load(f))
config.data.only_test = False
datamodule = GrandStaffDataset(config=config.data)

max_height, max_width = datamodule.train_set.get_max_hw()
max_len = datamodule.train_set.get_max_seqlen()

model_wrapper = SMT_Trainer(maxh=int(max_height), maxw=int(max_width), maxlen=int(max_len), 
                          out_categories=len(datamodule.train_set.w2i), padding_token=datamodule.train_set.w2i["<pad>"], 
                          in_channels=1, w2i=datamodule.train_set.w2i, i2w=datamodule.train_set.i2w, 
                          d_model=256, dim_ff=256, num_dec_layers=8)

AttributeError: 'Data' object has no attribute 'only_test'

In [ ]:
smt = SMT_Trainer.load_from_checkpoint("/workspace/weights/GrandStaff/GrandStaff_SMT_nianga.ckpt")

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
 
map_lm_to_smt(smt)
folder = "data/"
file = "sample.png"
image = cv2.imread(os.path.join(folder, file))


predictions, _ = smt.predict(convert_img_to_tensor(image).unsqueeze(0).to(device), 
                               convert_to_str=True)


#copy output to clipboard
text = "".join(predictions).replace('<b>', '\n').replace('<s>', ' ').replace('<t>', '\t').replace('**ekern_1.0','**kern')


print(text)
